In [ ]:
import cfg
import asyncio
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from utils import ZmqPairController, ShmStore
from functions import format_data_for_upload, GlobalSys, SysState

# --- 1. UI COMPONENT SETUP ---
style = {'description_width': 'initial'}

# Frequency and Span
freq_input = widgets.FloatText(value=98.0, description='Center Freq (MHz):', style=style)
span_input = widgets.FloatText(value=20.0, description='Span/Sample Rate (MHz):', style=style)

# Gain Sliders
lna_slider = widgets.IntSlider(value=16, min=0, max=40, step=8, description='LNA Gain (dB):', style=style)
vga_slider = widgets.IntSlider(value=24, min=0, max=62, step=2, description='VGA Gain (dB):', style=style)

# Acquisition Parameters
rbw_dropdown = widgets.Dropdown(
    options=[('1 kHz', 1000), ('10 kHz', 10000), ('100 kHz', 100000)],
    value=10000,
    description='RBW:',
    style=style
)

# Visual feedback
status_label = widgets.Label(value="Status: Ready")
plot_output = widgets.Output()

# Organize Layout (Mirroring your image)
row1 = widgets.HBox([freq_input, span_input])
row2 = widgets.HBox([lna_slider, vga_slider])
row3 = widgets.HBox([rbw_dropdown])
ui_layout = widgets.VBox([row1, row2, row3, status_label, plot_output])

# --- 2. MAIN LOGIC ---
async def run_interactive_realtime():
    log = cfg.set_logger()
    GlobalSys.set(SysState.REALTIME)
    controller = ZmqPairController(addr=cfg.IPC_ADDR, is_server=True, verbose=False)

    # Initialize Plot inside the Output Widget
    with plot_output:
        fig, ax = plt.subplots(figsize=(10, 5))
        line, = ax.plot([], [], color='#00ff00', lw=1)
        ax.set_ylim([-120, -10])
        ax.set_ylabel("Power (dBm)")
        ax.set_xlabel("Frequency (MHz)")
        ax.grid(True, alpha=0.3)
        plt.show()

    try:
        async with controller as zmq_ctrl:
            while True:
                # A. Build dynamic config from current Widget values
                current_state = {
                    "rf_mode": "realtime",
                    "center_freq_hz": int(freq_input.value * 1e6),
                    "sample_rate_hz": int(span_input.value * 1e6),
                    "rbw_hz": rbw_dropdown.value,
                    "lna_gain": lna_slider.value,
                    "vga_gain": vga_slider.value,
                    "span": int(span_input.value * 1e6),
                    "window": "hamming",
                    "scale": "dbm",
                    "overlap": 0.5,
                    "antenna_amp": True,
                    "antenna_port": 1,
                    "ppm_error": 0
                }

                start_time = time.perf_counter()

                # B. Communicate with C-Engine
                await zmq_ctrl.send_command(current_state)

                try:
                    raw_payload = await asyncio.wait_for(zmq_ctrl.wait_for_data(), timeout=2)
                    
                    # C. Process and Measure RTT
                    duration_ms = (time.perf_counter() - start_time) * 1000
                    data_dict = format_data_for_upload(raw_payload)
                    y_values = np.array(data_dict.get('data', []))

                    if len(y_values) > 0:
                        # D. Update Plot
                        center_mhz = freq_input.value
                        span_mhz = span_input.value
                        x_data = np.linspace(center_mhz - span_mhz/2, center_mhz + span_mhz/2, len(y_values))
                        
                        line.set_data(x_data, y_values)
                        ax.set_xlim([x_data[0], x_data[-1]])
                        
                        # E. Update UI Status
                        status_label.value = f"Status: Streaming | RTT: {duration_ms:.1f} ms | Bins: {len(y_values)}"
                        
                        fig.canvas.draw_idle()
                        fig.canvas.flush_events()

                except asyncio.TimeoutError:
                    status_label.value = "Status: ⚠️ C-Engine Timeout"

                # Brief sleep to allow Jupyter's event loop to handle widget interactions
                await asyncio.sleep(0.01)

    except Exception as e:
        status_label.value = f"Status: 🔥 Error: {e}"
    finally:
        GlobalSys.set(SysState.IDLE)

# --- 3. EXECUTION ---
display(ui_layout)

# Start the loop as a background task in Jupyter
loop = asyncio.get_event_loop()
if loop.is_running():
    # If in a notebook, create a task
    asyncio.create_task(run_interactive_realtime())
else:
    # If running as a standard script
    asyncio.run(run_interactive_realtime())